Importing necessary libraries

In [110]:
import pandas as pd 
import numpy as np
import re
import nltk
from keras.preprocessing.text import Tokenizer

unzip the source data

In [111]:
!unzip '/content/archive.zip' -d '/content/data'

Archive:  /content/archive.zip
replace /content/data/spam_ham.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/data/spam_ham.csv  


Pull the source data

In [112]:
dframe=pd.read_csv('/content/data/spam_ham.csv' , sep=',')
dframe[46:]

,text,sp_ha
46,From fork-admin@xent.com Wed Oct 9 10:55:12 20...,0
47,From razor-users-admin@lists.sourceforge.net F...,0
48,From fork-admin@xent.com Tue Jul 23 14:27:17 2...,0
49,From leadsales400042@yahoo.com Mon Jun 24 17:0...,1
50,From rssfeeds@jmason.org Thu Sep 26 16:34:06 2...,0
...,...,...
5823,From ilug-admin@linux.ie Mon Jul 22 18:12:45 2...,0
5824,From fork-admin@xent.com Mon Oct 7 20:37:02 20...,0
5825,Received: from hq.pro-ns.net (localhost [127.0...,1
5826,From razor-users-admin@lists.sourceforge.net T...,0


checking the rows

In [113]:
dframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5828 entries, 0 to 5827
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5826 non-null   object
 1   sp_ha   5825 non-null   object
dtypes: object(2)
memory usage: 91.2+ KB


below is the example for genuine email

In [114]:
dummy=dframe['text'][49]
dummy

'From leadsales400042@yahoo.com Mon Jun 24 17:03:55 2002 Return-Path: leadsales400042@yahoo.com Delivery-Date: Wed May 15 16:24:17 2002 Received: from MIRAFLORES.GOB.PE ([161.132.70.10]) by dogma.slashnull.org (8.11.6/8.11.6) with ESMTP id g4FFOAe12470 for <webmaster@efi.ie>; Wed, 15 May 2002 16:24:10 +0100 Message-Id: <200205151524.g4FFOAe12470@dogma.slashnull.org> Received: from yahoo.com([208.166.82.124]) by MIRAFLORES.GOB.PE (IBM OS/400 SMTP V05R01M00) with TCP; Wed, 15 May 2002 10:23:51 +0000 From: leadsales400042@yahoo.com To: webmaster@efi.ie Subject: Got Leads? Date: 15 May 2002 20:34:36 -0700 MIME-Version: 1.0 X-Keywords: Content-Type: text/html Content-Transfer-Encoding: quoted-printable <html> HEY, I AM FORWARDING YOU THE AD THEY SENT ME. I TRIED THE LEADS AND THINK WE SHOULD USE THEM FOR OUR TEAM. WHAT DO YOU THINK? THANKS, JOHN <head> <meta http-equiv=3D"Content-Type" content=3D"text/html; charset=3Dwindows= -1252"> <meta name=3D"GENERATOR" content=3D"Microsoft FrontPage 4

Example for spam email

In [115]:

ones=dframe['text'].where(dframe['sp_ha']=='1')
strngg=ones[1]

Preprocessing 

```
# Dropping data with nan values
```

---



---



In [116]:
dframe[dframe['sp_ha'].isna()]

,text,sp_ha
3872,From regal3@freeuk.com Fri Jul 26 03:51:44 200...,NaN
3873,NaN,NaN
3874,NaN,NaN


In [117]:
dframe=dframe.dropna()

In [118]:
dframe['sp_ha'].value_counts()

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            3911
1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

Removing values other than 0 or 1

In [119]:
dframe1=dframe[(dframe.sp_ha.isin(['0','1']))]
dframe1.sp_ha.value_counts()

0    3911
1    1912
Name: sp_ha, dtype: int64

Drop duplicate

In [120]:
dframe1.drop_duplicates(inplace=True)

<ipython-input-120-a39617d064bc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dframe1.drop_duplicates(inplace=True)


Refining text

In [121]:
strngg_2=re.sub(r'^.*?Subject', 'Subject', strngg)

In [122]:
strngg_3 = re.sub('http[^\s]+','',strngg_2)
strngg_3

'Subject: Cash in on your home equity Date: Tue, 04 Jun 2002 00:18:34 -1600 MIME-Version: 1.0 Content-Type: text/plain; charset="Windows-1252" X-Keywords: Content-Transfer-Encoding: 7bit Mortgage Lenders & Brokers Are Ready to compete for your business. Whether a new home loan is what you seek or to refinance your current home loan at a lower interest rate, we can help! Mortgage rates haven\'t been this low in years take action now! Refinance your home with us and include all of those pesky credit card bills or use the extra cash for that pool you\'ve always wanted... Where others say NO, we say YES!!! Even if you have been turned down elsewhere, we can help! Easy terms! Our mortgage referral service combines the highest quality loans with the most economical rates and the easiest qualifications! Take just 2 minutes to complete the following form. There is no obligation, all information is kept strictly confidential, and you must be at least 18 years of age. Service is available within

In [123]:
strngg_4=strngg_3.replace(r'^.*?','')

In [124]:
strngg_4

'Subject: Cash in on your home equity Date: Tue, 04 Jun 2002 00:18:34 -1600 MIME-Version: 1.0 Content-Type: text/plain; charset="Windows-1252" X-Keywords: Content-Transfer-Encoding: 7bit Mortgage Lenders & Brokers Are Ready to compete for your business. Whether a new home loan is what you seek or to refinance your current home loan at a lower interest rate, we can help! Mortgage rates haven\'t been this low in years take action now! Refinance your home with us and include all of those pesky credit card bills or use the extra cash for that pool you\'ve always wanted... Where others say NO, we say YES!!! Even if you have been turned down elsewhere, we can help! Easy terms! Our mortgage referral service combines the highest quality loans with the most economical rates and the easiest qualifications! Take just 2 minutes to complete the following form. There is no obligation, all information is kept strictly confidential, and you must be at least 18 years of age. Service is available within

In [125]:
from bs4 import BeautifulSoup

In [126]:
content = BeautifulSoup(dummy, "lxml")

In [127]:
dummy

'From leadsales400042@yahoo.com Mon Jun 24 17:03:55 2002 Return-Path: leadsales400042@yahoo.com Delivery-Date: Wed May 15 16:24:17 2002 Received: from MIRAFLORES.GOB.PE ([161.132.70.10]) by dogma.slashnull.org (8.11.6/8.11.6) with ESMTP id g4FFOAe12470 for <webmaster@efi.ie>; Wed, 15 May 2002 16:24:10 +0100 Message-Id: <200205151524.g4FFOAe12470@dogma.slashnull.org> Received: from yahoo.com([208.166.82.124]) by MIRAFLORES.GOB.PE (IBM OS/400 SMTP V05R01M00) with TCP; Wed, 15 May 2002 10:23:51 +0000 From: leadsales400042@yahoo.com To: webmaster@efi.ie Subject: Got Leads? Date: 15 May 2002 20:34:36 -0700 MIME-Version: 1.0 X-Keywords: Content-Type: text/html Content-Transfer-Encoding: quoted-printable <html> HEY, I AM FORWARDING YOU THE AD THEY SENT ME. I TRIED THE LEADS AND THINK WE SHOULD USE THEM FOR OUR TEAM. WHAT DO YOU THINK? THANKS, JOHN <head> <meta http-equiv=3D"Content-Type" content=3D"text/html; charset=3Dwindows= -1252"> <meta name=3D"GENERATOR" content=3D"Microsoft FrontPage 4

In [128]:
print(content.body.prettify())

<body>
 <p>
  From leadsales400042@yahoo.com Mon Jun 24 17:03:55 2002 Return-Path: leadsales400042@yahoo.com Delivery-Date: Wed May 15 16:24:17 2002 Received: from MIRAFLORES.GOB.PE ([161.132.70.10]) by dogma.slashnull.org (8.11.6/8.11.6) with ESMTP id g4FFOAe12470 for
  <webmaster>
   ; Wed, 15 May 2002 16:24:10 +0100 Message-Id: &lt;200205151524.g4FFOAe12470@dogma.slashnull.org&gt; Received: from yahoo.com([208.166.82.124]) by MIRAFLORES.GOB.PE (IBM OS/400 SMTP V05R01M00) with TCP; Wed, 15 May 2002 10:23:51 +0000 From: leadsales400042@yahoo.com To: webmaster@efi.ie Subject: Got Leads? Date: 15 May 2002 20:34:36 -0700 MIME-Version: 1.0 X-Keywords: Content-Type: text/html Content-Transfer-Encoding: quoted-printable  HEY, I AM FORWARDING YOU THE AD THEY SENT ME. I TRIED THE LEADS AND THINK WE SHOULD USE THEM FOR OUR TEAM. WHAT DO YOU THINK? THANKS, JOHN
   <meta charset="utf-8" content='3D"text/html;' http-equiv='3D"Content-Type"'/>
   <meta content='3D"Microsoft' frontpage="" name='3D"

In [129]:

job_elements = content.find_all("font")
for job_element in job_elements:
    title_element = job_element.find("font")

In [130]:
soup = BeautifulSoup(dummy, features="html.parser")
# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract() 

In [131]:
text = soup.get_text()

In [132]:
text

'From leadsales400042@yahoo.com Mon Jun 24 17:03:55 2002 Return-Path: leadsales400042@yahoo.com Delivery-Date: Wed May 15 16:24:17 2002 Received: from MIRAFLORES.GOB.PE ([161.132.70.10]) by dogma.slashnull.org (8.11.6/8.11.6) with ESMTP id g4FFOAe12470 for ; Wed, 15 May 2002 16:24:10 +0100 Message-Id: <200205151524.g4FFOAe12470@dogma.slashnull.org> Received: from yahoo.com([208.166.82.124]) by MIRAFLORES.GOB.PE (IBM OS/400 SMTP V05R01M00) with TCP; Wed, 15 May 2002 10:23:51 +0000 From: leadsales400042@yahoo.com To: webmaster@efi.ie Subject: Got Leads? Date: 15 May 2002 20:34:36 -0700 MIME-Version: 1.0 X-Keywords: Content-Type: text/html Content-Transfer-Encoding: quoted-printable  HEY, I AM FORWARDING YOU THE AD THEY SENT ME. I TRIED THE LEADS AND THINK WE SHOULD USE THEM FOR OUR TEAM. WHAT DO YOU THINK? THANKS, JOHN     LEAD SALE EXTENDED     YOU HAVE BEEN INVITED TO JOIN THE CO-OP   LEAD SALE = EXTENDED      We have= recently put together a lead co-op specifically for your group.\xa0

In [133]:
strngg_2=re.sub(r'^.*?Subject', 'Subject', text)
strngg_2

'Subject: Got Leads? Date: 15 May 2002 20:34:36 -0700 MIME-Version: 1.0 X-Keywords: Content-Type: text/html Content-Transfer-Encoding: quoted-printable  HEY, I AM FORWARDING YOU THE AD THEY SENT ME. I TRIED THE LEADS AND THINK WE SHOULD USE THEM FOR OUR TEAM. WHAT DO YOU THINK? THANKS, JOHN     LEAD SALE EXTENDED     YOU HAVE BEEN INVITED TO JOIN THE CO-OP   LEAD SALE = EXTENDED      We have= recently put together a lead co-op specifically for your group.\xa0 We have = been able to lock the price at as little as 50 cents per lead!!\xa0 = Leads are normally $1-$3 per...     \xa0= ;    So...= .  SIGN = UP TODAY....  GET O= UR LEADS....  AND G= ET TO WORK!!      Here is what we hav= e to offer you:         NETWORK MARKETER OR GIFTER CONTACTS  All Leads Guaranteed To Be Network Marketers / o= r Gifters Guaranteed To Be Current, Active and Experienced=  Leads, Have Either Responded To A Internet Quest= ionnaire, Or Are Already On Our Contact List, Of Current, Active Networkers, Open To Oppo

In [134]:
my_new_string = re.sub('[^a-zA-Z0-9 \n\.]', '', strngg_2)
my_new_string

'Subject Got Leads Date 15 May 2002 203436 0700 MIMEVersion 1.0 XKeywords ContentType texthtml ContentTransferEncoding quotedprintable  HEY I AM FORWARDING YOU THE AD THEY SENT ME. I TRIED THE LEADS AND THINK WE SHOULD USE THEM FOR OUR TEAM. WHAT DO YOU THINK THANKS JOHN     LEAD SALE EXTENDED     YOU HAVE BEEN INVITED TO JOIN THE COOP   LEAD SALE  EXTENDED      We have recently put together a lead coop specifically for your group. We have  been able to lock the price at as little as 50 cents per lead  Leads are normally 13 per...          So... .  SIGN  UP TODAY....  GET O UR LEADS....  AND G ET TO WORK      Here is what we hav e to offer you         NETWORK MARKETER OR GIFTER CONTACTS  All Leads Guaranteed To Be Network Marketers  o r Gifters Guaranteed To Be Current Active and Experienced  Leads Have Either Responded To A Internet Quest ionnaire Or Are Already On Our Contact List Of Current Active Networkers Open To Opportunity. Leads IncludeAt Least Name Phone And Email          US

In [135]:
" ".join(my_new_string.split())

'Subject Got Leads Date 15 May 2002 203436 0700 MIMEVersion 1.0 XKeywords ContentType texthtml ContentTransferEncoding quotedprintable HEY I AM FORWARDING YOU THE AD THEY SENT ME. I TRIED THE LEADS AND THINK WE SHOULD USE THEM FOR OUR TEAM. WHAT DO YOU THINK THANKS JOHN LEAD SALE EXTENDED YOU HAVE BEEN INVITED TO JOIN THE COOP LEAD SALE EXTENDED We have recently put together a lead coop specifically for your group. We have been able to lock the price at as little as 50 cents per lead Leads are normally 13 per... So... . SIGN UP TODAY.... GET O UR LEADS.... AND G ET TO WORK Here is what we hav e to offer you NETWORK MARKETER OR GIFTER CONTACTS All Leads Guaranteed To Be Network Marketers o r Gifters Guaranteed To Be Current Active and Experienced Leads Have Either Responded To A Internet Quest ionnaire Or Are Already On Our Contact List Of Current Active Networkers Open To Opportunity. Leads IncludeAt Least Name Phone And Email USE US AS A TOOL TO BUILD YOUR BUSINESSnbs p REFER AS MANY 

In [136]:
# re.sub('<[^>]+>', '', s)
result = strngg_2.rsplit('printable', 1)[1]
result

'  HEY, I AM FORWARDING YOU THE AD THEY SENT ME. I TRIED THE LEADS AND THINK WE SHOULD USE THEM FOR OUR TEAM. WHAT DO YOU THINK? THANKS, JOHN     LEAD SALE EXTENDED     YOU HAVE BEEN INVITED TO JOIN THE CO-OP   LEAD SALE = EXTENDED      We have= recently put together a lead co-op specifically for your group.\xa0 We have = been able to lock the price at as little as 50 cents per lead!!\xa0 = Leads are normally $1-$3 per...     \xa0= ;    So...= .  SIGN = UP TODAY....  GET O= UR LEADS....  AND G= ET TO WORK!!      Here is what we hav= e to offer you:         NETWORK MARKETER OR GIFTER CONTACTS  All Leads Guaranteed To Be Network Marketers / o= r Gifters Guaranteed To Be Current, Active and Experienced=  Leads, Have Either Responded To A Internet Quest= ionnaire, Or Are Already On Our Contact List, Of Current, Active Networkers, Open To Opportunity. Leads Include:\xa0\xa0\xa0\xa0[At Least]= Name, Phone, And Email  \xa0        ***USE US AS A TOOL TO BUILD YOUR BUSINESS!!&nbs= p; REFER AS M

In [137]:
strngg_2=re.sub(r'^.*?Subject', 'Subject', text)
my_new_string = re.sub('[^a-zA-Z0-9 \n\.]', '', strngg_2)
my_new_string_2=''.join(my_new_string.split('.'))
my_new_string_2=" ".join(my_new_string_2.split())
result = my_new_string_2.rsplit('printable', 1)[1]
result=result.lower()
result

' hey i am forwarding you the ad they sent me i tried the leads and think we should use them for our team what do you think thanks john lead sale extended you have been invited to join the coop lead sale extended we have recently put together a lead coop specifically for your group we have been able to lock the price at as little as 50 cents per lead leads are normally 13 per so sign up today get o ur leads and g et to work here is what we hav e to offer you network marketer or gifter contacts all leads guaranteed to be network marketers o r gifters guaranteed to be current active and experienced leads have either responded to a internet quest ionnaire or are already on our contact list of current active networkers open to opportunity leads includeat least name phone and email use us as a tool to build your businessnbs p refer as many people as you can earn a true residual income by the duplication of your team cause remember remember your system now includes a proven lead system remem

In [138]:
from bs4 import BeautifulSoup
def format(string):
  soup = BeautifulSoup(string, features="html.parser")
# kill all script and style elements
  for script in soup(["script", "style"]):
    script.extract()
  text=soup.get_text() 
  my_new_string_1=re.sub(r'^.*?Subject', 'Subject', text)
  my_new_string_2=re.sub(r'[A-Za-z0-9]*@[A-Za-z]*\.?[A-Za-z0-9]*', "", my_new_string_1)
  my_new_string_3 = re.sub('[^a-zA-Z0-9 \n\.]', '', my_new_string_2)
  my_new_string_4=''.join(my_new_string_3.split('.'))
  my_new_string_5=" ".join(my_new_string_4.split())
  my_new_string_6=re.sub(r'[A-Za-z0-9]*@[A-Za-z]*\.?[A-Za-z0-9]*', "", my_new_string_5)
  my_new_string_7 = re.sub('http[^\s]+','',my_new_string_6)
  # my_new_string_5 = my_new_string_4.rsplit('printable', 1)[1]
  output_string=my_new_string_7.lower()
  return output_string

In [139]:
r = format(dummy)
r

'subject got leads date 15 may 2002 203436 0700 mimeversion 10 xkeywords contenttype texthtml contenttransferencoding quotedprintable hey i am forwarding you the ad they sent me i tried the leads and think we should use them for our team what do you think thanks john lead sale extended you have been invited to join the coop lead sale extended we have recently put together a lead coop specifically for your group we have been able to lock the price at as little as 50 cents per lead leads are normally 13 per so sign up today get o ur leads and g et to work here is what we hav e to offer you network marketer or gifter contacts all leads guaranteed to be network marketers o r gifters guaranteed to be current active and experienced leads have either responded to a internet quest ionnaire or are already on our contact list of current active networkers open to opportunity leads includeat least name phone and email use us as a tool to build your businessnbs p refer as many people as you can ear

In [140]:
df2 = dframe1['text'].apply(format)
# dummy=dframe['text'][52]
# dummy

In [141]:
df2[901]

'subject world wide words 20 jul 02 to org world wide words issue 296 saturday 20 july 2002 sent each saturday to 15000 subscribers in at least 119 countries editor michael quinion thornbury bristol uk issn 14701448 if you respond to this mailing remember to change the outgoing address to one of those in the contact addresses section contents 1 turns of phrase beanpole family 2 weird words jobbernowl 3 beyond words 4 qa chip off the old block bells and whistles mortarboard 5 endnote 6 subscription commands 7 contact addresses 1 turns of phrase beanpole family historically families have usually had more children than parents resulting in family trees that looked like pyramids however in recent years especially in countries like britain and the us the number of children per generation has steadily gone down while life span has increased this has led to a shape of family tree that some researchers have likened to a beanpole tall and thin with few people in each generation the term beanpol

Removing Stopwords

In [142]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [143]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))
", ".join(stopwords.words('english'))
def remove_stopwords(text):
    return " ".join([word for word in text.split() if word not in stop_words])

In [144]:
df2

0       subject re ilug architecture crossover trouble...
1       subject cash in on your home equity date tue 0...
2       subject are your mortgage rates the best they ...
3       subject online approval for 5000 now mimeversi...
4       subject re ilug expanding a string multiple ti...
                              ...                        
5823    subject re ilug nmap results date mon 22 jul 2...
5824    subject re the absurdities of life inreplyto 3...
5825    subject real drugsviagra and phentrimine date ...
5826    subject razorusers no positive razor hits send...
5827    subject are you being freeserved date mon 30 s...
Name: text, Length: 5320, dtype: object

In [145]:
df2 = df2.apply(lambda x: remove_stopwords(x))

In [146]:
df2[901]

'subject world wide words 20 jul 02 org world wide words issue 296 saturday 20 july 2002 sent saturday 15000 subscribers least 119 countries editor michael quinion thornbury bristol uk issn 14701448 respond mailing remember change outgoing address one contact addresses section contents 1 turns phrase beanpole family 2 weird words jobbernowl 3 beyond words 4 qa chip old block bells whistles mortarboard 5 endnote 6 subscription commands 7 contact addresses 1 turns phrase beanpole family historically families usually children parents resulting family trees looked like pyramids however recent years especially countries like britain us number children per generation steadily gone life span increased led shape family tree researchers likened beanpole tall thin people generation term beanpole family around academic literature least since 1987 rarely appears elsewhere recent british report brought wider public notice least uk researchers find slangy prefer jargon term verticalised describe fam

Tokenization and converting to sequences

In [147]:
# tokenize text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df2)

word_index = tokenizer.word_index
vocab_size = len(word_index)
vocab_size

68591

Convert whole text into uniform list of features

In [148]:
sequences = tokenizer.texts_to_sequences(df2)

In [149]:
!pip install pad_sequences
import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [150]:
len_of_seq=max(len(data) for data in df2)
len_of_seq

23852

In [151]:
padded_seq = pad_sequences(sequences, maxlen = 23852,padding='post',truncating='post')

In [152]:
padded_seq[0]

array([   2,   48, 2899, ...,    0,    0,    0], dtype=int32)

In [153]:
padded_seq.shape

(5320, 23852)